In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/meta
/kaggle/input/file.txt
/kaggle/input/test
/kaggle/input/train


# Transforming data

In [2]:
import torch 
import torchvision


transform_train = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),  # Resize to 256x256
    #torchvision.transforms.CenterCrop(64),  # Crop a central 224x224 region
    #torchvision.transforms.RandomResizedCrop(size=(64,64), scale=(0.8, 1.0)),  # Randomly resize and crop
    torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip horizontally
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    torchvision.transforms.ToTensor(),  # Convert to PyTorch tensor
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ImageNet
])

transform_test=torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),  
    torchvision.transforms.ToTensor(),  # Convert to PyTorch tensor
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Importing and Loading Data 

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR100

root="./data"
train = CIFAR100(root=root, transform=transform_train,download=True,train=True)
test= CIFAR100(root=root, transform=transform_test,download=True,train=False)

Files already downloaded and verified
Files already downloaded and verified


# Initialize Dataloader

In [4]:
train_loader = DataLoader(train, batch_size=100, shuffle=True)
test_loader= DataLoader(test, batch_size=16, shuffle=True)

In [5]:
len(train_loader)

500

# Importing torch

In [6]:
import torch.nn as nn


'\nclass VGG16(nn.Module):\n    def __init__(self, num_classes=10):\n        super(VGG16, self).__init__()\n        self.layer1 = nn.Sequential(\n            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),\n            nn.BatchNorm2d(64),\n            nn.ReLU())\n        self.layer2 = nn.Sequential(\n            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),\n            nn.BatchNorm2d(64),\n            nn.ReLU(), \n            nn.MaxPool2d(kernel_size = 2, stride = 2))\n        self.layer3 = nn.Sequential(\n            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),\n            nn.BatchNorm2d(128),\n            nn.ReLU())\n        self.layer4 = nn.Sequential(\n            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),\n            nn.BatchNorm2d(128),\n            nn.ReLU(),\n            nn.MaxPool2d(kernel_size = 2, stride = 2))\n        self.layer5 = nn.Sequential(\n            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),\n            nn.

# Hyperparameter tuning and  Defining Model Architecture

In [7]:
import torchvision.models as models
device='cuda'
num_classes = 100
num_epochs = 2
learning_rate = 0.001
model = models.vgg16(weights='VGG16_Weights.DEFAULT')
#model=nn.DataParallel(model)


# Replace the last classifier layer
num_features_in = model.classifier[-1].in_features  # Get input size of the last layer
model.classifier[-1] = torch.nn.Linear(num_features_in,100)
model=model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.7)  


# Train the model
total_step = len(train_loader)


In [8]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=100, bias=True)
)

In [9]:
model.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size

# Traning and Testing loop

In [10]:
from tqdm import tqdm
total_step = len(train_loader)
patience=2
patience_count=0
minimum_loss=float('inf')

for epoch in range(num_epochs):
    
    for  images, labels in tqdm(train_loader):  
        correct = 0
        total = 0
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        model.train()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Loss {loss.item()}" )
    print('Accuracy of the network on the {}  images: {} %'.format(50000, 100 * correct / total))
        
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        eval_loss=0
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            eval_loss+=criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        print(f" Loss Minimum {minimum_loss} Loss_current {eval_loss}")
            
        if eval_loss<=minimum_loss:
            patience_count=0
            minimum_loss=eval_loss
        else:
            patience_count+=1
            print(f"Patience Counter {patience_count}")
        print('Accuracy of the network on the {} validation images: {} %'.format(10000, 100 * correct / total))
        
        if patience_count>=patience:
            print(f"Early stopping initialized")
            break


100%|██████████| 500/500 [07:40<00:00,  1.09it/s]


Loss 1.8397932052612305
Accuracy of the network on the 5000  images: 45.0 %


100%|██████████| 625/625 [00:38<00:00, 16.27it/s]


 Loss Minimum inf Loss_current 1103.7502732276917
Accuracy of the network on the 5000 validation images: 50.25 %


100%|██████████| 500/500 [07:38<00:00,  1.09it/s]


Loss 1.3759379386901855
Accuracy of the network on the 5000  images: 59.0 %


100%|██████████| 625/625 [00:38<00:00, 16.26it/s]

 Loss Minimum 1103.7502732276917 Loss_current 891.5986252427101
Accuracy of the network on the 5000 validation images: 58.96 %


# Passing Radnom image to find Cnn output

In [14]:
model = model.to(device)
a=torch.randn(1,3,224,224)
a=a.to(device)
#center_crop = torchvision.transforms.CenterCrop(64)

# Apply the center cropping transform to the input image
#a = center_crop(a)

x=model(a)
x.shape

torch.Size([1, 100])